# Recomendador para series de anime. Entrega 2
__Emilio Lizama, Alonso Reyes, Juan-Pablo Silva__

## 1. Introducción

En la última década, la cultura japonesa ha a comenzado a despertar un interes creciente en occidente. Cada vez más gente está interesada en visitar Japón, leer su literatura, y como no, ver sus series de televisión animadas: anime.
El anime es un término usado para referirse a la animación japonesa, la cual puede tener tematicas infantiles, acción o temas más complejos, dirigidos a audiencias adultas.

Aun así con todo este aumento de personas interesadas y viendo anime, no existe alguna página web o aplicación que analice gustos y trate de recomendar series que probablemente serán de agrado de quien las buscó. Actualmente las páginas más populares para llevar registro de qué anime se han visto solo recomiendan anime de forma general por sus puntuaciones totales, o los clásicos "porque viste A te podría gustar B", cosas que son calculos básicos hechos acorde a los distintos géneros de cada serie y si coinciden con otras, de preferencia de mayor puntuación. Es por esto que nosotros en el proyecto crearemos un sistema recomendador, que actúe como un predictor sobre si a cierto usuario le gustará o no cierto anime, y podrá generar recomendaciones personalizadas entorno a registros sobre cada usuario.

En esta segunda entrega presentamos nuestros avances respecto al modelo recomendador capaz de recomendar series de anime personalizadamente.

## 2. Entrega 1

Para nuestra entrega pasada mostramos los datos que usariamos e hicimos un análisis exploratorio de los datos mismos. A modo de resumen, tenemos los datos te 13.510 anime, junto con sus puntuaciones y genemos. Además, tambien tenemos los anime vistos y las puntuaciones otorgadas por 4.140 usuarios, lo cual suma más de 1 millón de calificaciones. Los datos fueron obtenidos desde __[MyAnimeList](https://myanimelist.net/)__, una página que contiene una base de datos sobre todos los anime, donde sus usuarios son capaces de guardar en sus "listas" los anime que han visto y asignarles un puntaje del 1 al 10.

## 3. Metodología y pruebas

Para esta etapa del proyecto, ahora que tenemos conocimiento de como nuestros datos estan organizados, y las variaciones que existe, fue momento de explorar posibles algoritmos, modelos y libreria que pudieran ayudarnos a modelar nuestros datos de forma que pudiera precedir gustos.

Para avanzar en el proyecto tomamos un par de direcciones que finalmente creemos pueden combinarse en un modelo mas poderoso al final. Nuestra primera hipotesis es que si pudieramos clasificar usuarios en distintos grupos, podriamos hacer recomendaciones en base a los anime que X usuario no haya visto, pero sí hayan visto, y les hayan gustado, a otros miembros del grupo. Esto se presenta en la siguientes secciones.

### 3.1 Dimensionalidad

El primer problema con el que chocamos rapidamente al proponernos clasificar usuarios, fue la dimensionalidad de nuestros datos. Con más de 13 mil anime, se tienen más de 13 mil dimensiones que considerar. Además, la matriz de interaccion usuario|anime es sumamente dispersa, ya que los usuarios en promedio han visto menos de 200 anime.

Para datos dispersos como los nuestros, aplicar PCA no trae los mismos resultados que traería para una matriz más densa. Por esta razón, una solución que usamos fue LSA (latent semantic analysis) que nos entrega una matriz, con una dimension que nosotros especificamos. Esto es principalmente porque LSA no centra los datos antes de computar el SVD (singular value descomposition). Esto permite que funcione mucho más rápido con la representación de matrices dispersas de scipy.

Dicho esto, no hicimos mayores análisis con las matrices reducidas ya que primero intentamos ver realmente qué tanto afectarían nuestros resultados tener tan alta dimensionalidad.


### 3.2 Clustering (K-means)

Uno de los primeros pasos que tomamos fue usar KMeans para clasificar nuestros datos, en este caso, para clasificar usuarios. Definimos a un usuario por el conjunto de anime que ha visto, es decir, cada usuario tiene un vector de 13 dimensiones donde cada dimension corresponde a un anime. Luego en la posicion de cada anime, se inserta la puntuación que ese usuario le dió a ese anime. De esta manera podemos hubicar a cada usuario como un punto en el espacio 13.510 dimensional y calcular distancias a partir de este.

En este contexto se hicieron 2 pruebas. Una con las 13 mil dimensiones, es decir con los datos originales, y otra con una reducción de dimensiones de 13 mil a mil. El porqué se redujo a mil es porque creemos que si bien 13 mil anime es demasiado, y nadie los ha visto todos, y muy buena parte de ellos tampoco son populares, mil anime no es demasiado. Hay muchos usuarios que han visto más de 1.000 series, por lo que consideramos que tener mil valores representativos es una buena medida.

Como no sabemos realmente cuantos grupos de usuarios queremos, usamos el elbow method, que se basa en probar una serie de "K"s, número de clusters, luego calcular la inercia, o la suma del error cuadrático, y graficar este error decrecer. La idea es escojer el K que hace disminuir más el error antes de que este empiece a disminuir menos abruptamente. Como tenemos datos tan dispersos, cláramente el error no se estabilizará fácilmente, pero podrá dar una idea de qué valor de K es uno útil para nuestro análisis.

Primero leemos los datos desde el archivo de relaciones que creamos, el cual es simplemente el id del usuario junto con un vector de 13 mil dimensiones donde cada valor del vector corresponde al puntaje que le dió el usuario a ese anime (el indice corresponde al id del anime), y es 0 si este no lo ha visto o puntuado.

In [ ]:
import csv

import matplotlib.pyplot as plt
import numpy as np
from scipy import sparse
from sklearn.decomposition import TruncatedSVD

data = list()
with open('relations.csv', 'r') as f:
    ff = csv.reader(f)
    for line in ff:
        if line:
            data.append([int(i) for i in line[1:]])

data = sparse.coo_matrix(np.array(data))

# Descomentar para reducir dimensionaldad y utilizar reduced_data
# svd = TruncatedSVD(n_components=1000, n_iter=10, random_state=0)
# reduced_data = svd.fit_transform(data)

Ahora creamos los gráficos para encontrar el K adecuado al problema.

In [ ]:
from sklearn.cluster import KMeans

max_k = 1001 # cambiar 1001 -> 501 para reduced_data

sse = []
for k in range(1, max_k, 50): 
    print("K: {}".format(k))
    kmeans = KMeans(n_clusters=k, random_state=0).fit(data) # reduced_data.tocsr())
    sse.append(kmeans.inertia_)

plt.plot([i for i in range(1, max_k, 50)], sse, 'o-')
plt.show()

Debido a que esto toma mucho tiempo, insertaremos las imagenes directamente
![title](elbow_kmeans.png)
![title](elbow_kmeans_lsa.png)

Como se puede ver, el error no deja de disminuir. Esto es lógico ya que el error se hace 0 solo cuando K=numero de elementos. Respaldado parcialmente por los gráficos, decidimos utilizar un K=50, con el cual KMeans le asignó a cada usuario un grupo. En una siguiente sección se hablará de la evaluación de estos resultados, pero inspecciones manuales sobre algunos grupos de clusters selectos resultó en agrupaciones correctas de usuarios con gustos similares. Por supuesto, este tipo de evaluación no tiene respaldo matemático, pero es una señal de que el método en efecto tiene resultados.

### 3.3 Ki-Means

Basado en K-means, Ki-means es nuestra implementación del algoritmo para hacer clustering sobre datos dispersos. Nuestra hipotesis para hacer esto es la siguiente: asumamos existen solo 3 anime tal que cada anime es representado por su índice en un vector de 3 valores. Digamos Mike puso un 7 al primer anime, no ha visto el segundo y puso un 1 al tercero, esto seria (7, 0 ,1). Luego tenemos a Pablo que puso los siguientes puntajes (7, 10 , 0), es decir no ha visto el tercero. Finalmente esta Alejando con (2, 10, 5). En un escenario como este K-means con K=2 clasificaria juntos a Pablo y Alejandro, ya que su distancia euclideana es efectivamente menor. Sin embargo, nosotros proponemos que los que realmente se parecen más son Mike con Pablo, ya que no tenemos información sobre si a Mike le gustará o no el segundo anime, y tampoco tenemos información sobre el 3er anime para Pablo, pero ambos pusieron un 7 al primer anime, por lo que su distancia sería 0.

Es basado en esta hipótesis que implementamos K-means de forma que considerara solo variables comunes entre puntos, salvo el centroide, en el cual se tomaban todas sus variables en cuenta. En pocas palabras esta implementación no considera la variables que tiene 0 en los puntos del dataset, es decir, no considera los anime que los usuarios no han visto cuando interactuan entre ellos. El código de la implementación se encuentra en __[GitHub](https://github.com/juanpablos/anime-recommender/blob/master/src/ki_means.py)__. Lamentablemente no pudimos hacer evaluaciones con esta variación de K-means ya que las dimensiones y usuarios eran demasiados, y el algoritmo fue implementado muy ingenuamente, haciendolo demasiado lento para las evaluaciones y calculos necesarios. Sin embargo, pensamos que si el algoritmo pudiese optimizarse, es posible obtener mejores resultados para el clustering.

### 3.4 Factorization Machine

Las Factorization Machines son un concepto relativamente nuevo introducido el 2010 por Steffen Rendle [1]. Las FM son un modelo que permite simular la gran mayoría de los modelos de factorización presentes en feature engineering, que básicamente es entender sobre el dominio en que se esta trabajando para crear características para entrenar modelos de machine learning. Según su mismo inventor, las FM son comparables a los SVMs (support vector machine), que son usados para clasificación.

La parte importante que se aplica a nuestro problema de recomendar series de anime, es que la FM pueden considerar más que solo una matriz de interacción. La mayoría de los sistemas recomendadores toman una matriz (usuario, item, puntuación), pero hay veces en las que tambien se tienen datos externos a solo esas interacciones, como pueden ser tags, géneros de música, pre clasificación de usuarios, etc. Este es justamente nuestra situación, tenemos géneros, múltiples, para cada anime. Además podríamos considerar edad, sexo y pais de residencia de los usuarios a evaluar, lo cual se convierte en más carácteristicas extra, que llamaremos metadata.

Aquí utilizamos LightFM [2], una librería de Python para trabajar con factorization machines que implementa un modelo híbrido con feedback explicito e implicito. A continuación se presenta cómo usamos este modelo y un output de ejemplo. También se puede encontrar en El código completo usado por nosotros se puede encontrar en __[GitHub](https://github.com/juanpablos/anime-recommender/blob/master/src/factorized_machine.py)__.

Librerias requeridas

In [ ]:
import numpy as np
import csv
import ast
import time
from scipy import sparse
from lightfm import LightFM
from lightfm.evaluation import auc_score

Se crean mapeos a ids y nombres especificos al proyecto. Principalmente porque en MyAnimeList los id de los anime tienen un valor incremental pero muchas veces se saltan valores, un ejemplo es que hay 13 mil anime en los datos, pero los id superan el 30 mil.

In [ ]:
anime_id_name = {}
new_id_dict = {}

with open('new_anime_id.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    for line in reader:
        new_id_dict[line[0]] = line[1]

with open('general.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    for line in reader:
        if line[0] == 'anime_id':
            continue
        anime_id_name[new_id_dict[line[0]]] = line[1]

El vector de metadata para esta primera prueba son solo los géneros. Estos corresponden a una matriz de (numero de anime, numero de géneros), lo cual nuevamente es una matriz dispersa, ya que cada indice de columna corresponde a un género distinto, el valor es 0 si ese anime no es de ese género, 1 si sí lo es.

In [ ]:
item_features = np.zeros((13510, 44))

with open('anime_dict.csv', 'r') as fm:
    ff = csv.reader(fm)
    for line in ff:
        a_id = int(line[0])
        a_genres = ast.literal_eval(line[1])
        for g in a_genres:
            item_features[(a_id-1, g-1)] = 1

# Leemos los datos de interaccion entre usuario/anime
data = list()
with open('out_file.csv', 'r') as f:
    ff = csv.reader(f)
    for line in ff:
        if line:
            data.append([int(i) for i in line[1:]])

data2 = np.array(data)

A continuación se muestra el código del modelo. Se explicará con comentarios la utilidad de cada linea.

In [ ]:
# Creamos los dataset para entrenar y otro para evaluar el modelo
train = np.copy(data2)
test = np.copy(data2)

for i in range(int(len(data2) * 0.8)):
    for j in range(len(test[i])):
        test[(i,j)] = 0.

for i in range(int(len(data2) * 0.8), int(len(data2))):
    for j in range(len(train[i])):
        train[(i,j)] = 0.

# Parametros del modelo
NUM_THREADS = 4
NUM_EPOCHS = 30

# Convertimos los arreglos de numpy en matrices dispersas de scipy.
# Esto permite una manipulación más eficiente de estas por el modelo.
train = sparse.csr_matrix(train)
test = sparse.csr_matrix(test)
item_features = sparse.csr_matrix(item_features)

# Se usa la función de perdida WARP (Weighted Approximate-Rank Pairwise), que en general funciona mejor que
# la mas comun BPR  (Bayesian Personalised Ranking)
model = LightFM(loss='warp', learning_rate=0.05)

# Damos al modelo la sparse matrix (N_usuarios, N_anime) de interacciones para entrenar, junto
# a la sparse matrix de los generos (N_anime, N_generos)
model.fit(interactions=train, epochs=NUM_EPOCHS, item_features=item_features, num_threads=NUM_THREADS)

# Hacemos predicciones para el primer usuario (id=0) que ha visto el conjunto data2[0] de anime. Luego seleccionamos
# los top 5 animes que recomienda el modelo.
predictions = model.predict(0, data2[0]).argsort()[-5:][::-1]

for p in predictions:
    print(anime_id_name[str(p+1)])

# Evaluamos el modelo respecto al set de training
train_auc = auc_score(model, train, item_features=item_features, num_threads=NUM_THREADS).mean()
print('Collaborative filtering train AUC: %s' % train_auc)

# Evaluamos el modelo respecto al set de testing
test_auc = auc_score(model, test, train_interactions=train, item_features=item_features, num_threads=NUM_THREADS).mean()
print('Collaborative filtering test AUC: %s' % test_auc)

> Kimi no Na wa.
> Gin no Guardian
- Mutsugo to Ouma no Monogatari
- Jitsu wa Watashi wa
- Elmer no Bouken: My Father's Dragon
- Collaborative filtering train AUC: 0.7323
- Collaborative filtering test AUC: 0.673484

## 4. Evaluacion


## 5. Conclusiones y acciones futuras

## Referencias
[1] Rendle, S. (2010). Factorization Machines.. In G. I. Webb, B. Liu, C. Zhang, D. Gunopulos & X. Wu (eds.), ICDM (p./pp. 995-1000), : IEEE Computer Society. ISBN: 978-0-7695-4256-0 

[2] Kula, M. (2015). Metadata Embeddings for User and Item Cold-start Recommendations.. In T. Bogers & M. Koolen (eds.), CBRecSys@RecSys (p./pp. 14-21), : CEUR-WS.org. 